In [1]:
### Import libraries
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.feature_selection import RFE
from sklearn import cross_validation
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
%pylab
from sklearn.pipeline import make_pipeline
from scipy.signal import argrelextrema
import statsmodels.discrete.discrete_model as sm

/home/deva/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
### Import Data Files
df_satData = pd.read_csv('distSatellite_withLabels.csv')

In [3]:
df_satData.head(2)

,DISTRICT,101,DN_00,DN_01,DN_02,DN_03,DN_04,DN_05,DN_06,DN_07,...,CHH_3,FC_1,FC_2,FC_3,BF_1,BF_2,BF_3,EMP_1,EMP_2,EMP_3
0,Adilabad,532,5237.12,0,0,0.76,1815.67,3332.38,2043.90,1049.05,...,0,1,0,0,1,0,0,0,1,0
1,Agra,146,401.66,0,0,0.00,48.54,451.36,620.77,441.57,...,0,0,0,1,0,0,1,1,0,0


In [4]:
### List of labels
lis_labelsAll = ['MSL_1','MSL_2','MSL_3','MSW_1','MSW_2','MSW_3','CHH_1','CHH_2','CHH_3','FC_1','FC_2','FC_3','BF_1','BF_2','BF_3','EMP_1','EMP_2','EMP_3']

### List of useful Labels
lis_labels= ['EMP_1','EMP_2','EMP_3']

## Building features from night light data

In [5]:
var = []
for i in range(64):
    var.append('DN_' + str(i).zfill(2))

df_satData['Area'] = 0
for i in var:
    df_satData['Area']=df_satData['Area'] + df_satData[i]
df_satData['Area'] = df_satData['Area'].replace(0,1)

for i in var:
    df_satData['p'+i] = (df_satData[i])/(df_satData['Area']*1.0)

for i,j in enumerate(var):
    df_satData['q'+j] = df_satData[j]*(i+1)

var3 = []
for i in var:
    var3.append('p'+i)
#for i in var:
#    var1.append(i)
#for i in var:
#    var1.append('r'+i)
#for i in var:
#    var1.append('rp'+i)

for i in var:
    var3.append('q'+i)
#for i in var:
#    var1.append('rq'+i)

## Appling Pca to matrix formed from features of nightlight data

In [6]:
X = df_satData.loc[:,var3].values

X = preprocessing.scale(X)

pca= PCA(svd_solver='full')
pca.fit(X, y=None )
a= pca.explained_variance_ratio_
for i in range(len(a)):
    a[i]=round(a[i],3)
    #print(a)
#print(pd.DataFrame(pca.components_))
###Creating the array for clustering using transformed variables

In [7]:
 0.434 +0.171+ 0.082+0.044 + 0.033 + 0.023 + 0.022 + 0.02+   0.016 + 0.013

0.8580000000000001

In [8]:
X1=pca.fit_transform(X)
X1=X1[:,0:8]
#X=normalize(X,axis=0)

In [9]:
np.shape(X1)
[m,k] = np.shape(X1)

col_X1 = []
for j in range(k):
    col_X1.append('pca_'+str(j))

In [10]:
df_pcaNl = pd.DataFrame(X1,columns=col_X1)

In [11]:
df_pcaNl

,pca_0,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7
0,2.730040,-5.321751,-0.860339,3.246664,2.694576,0.809633,1.399437,-1.985949
1,8.599441,-1.433546,-0.258951,-0.041424,-0.715240,0.191611,-1.324506,1.446709
2,12.113937,-4.310023,-7.136619,-2.049438,1.315599,1.148854,-0.142953,-1.886010
3,9.380082,-9.838996,1.143180,7.110819,1.978624,0.743656,3.220930,-1.526049
4,-4.989618,0.675239,-0.595747,-1.363547,-0.947617,-0.620833,0.649545,0.154708
5,7.498983,-3.553394,-3.680371,-1.386795,0.034324,-0.370543,0.313855,0.138722
6,-1.941979,-1.328051,-1.370892,1.666265,3.096619,1.158195,-1.128704,-1.105413
7,1.507920,-0.038659,10.026218,-1.014820,-2.043778,1.509291,-0.294906,-0.862596
8,-0.511985,-0.142570,-1.520302,0.314494,0.451678,0.041638,-0.987478,0.768731
9,-5.649400,0.716519,-0.501431,-1.259249,-1.017083,-0.447669,0.562549,-0.021862


## Building features out of Modis Data

In [12]:
df_satData['Urban and built-up']=df_satData['Urban and built-up']+1
df_satData['CropRatio']=(df_satData['Croplands']+df_satData['Cropland/Natural vegetation mosaic'])/df_satData['Area']
df_satData['AvgUrbanNTL']=df_satData['sum']/df_satData['Urban and built-up']
df_satData['UrbanRatio']= df_satData['Urban and built-up']/df_satData['Area']

df_satData['Natural']= 0
for i in ['Water','Evergreen Needleleaf forest','Evergreen Broadleaf forest','Deciduous Needleleaf forest','Deciduous Broadleaf forest','Mixed forest','Closed shrublands','Open shrublands','Woody savannas','Savannas','Grasslands','Permanent wetlands','Snow and ice']:
    df_satData['Natural']=df_satData['Natural']+df_satData[i]
    
df_satData['CropRemainRatio']=(df_satData['Croplands']+df_satData['Cropland/Natural vegetation mosaic'])/(df_satData['Area']-df_satData['Natural'])
df_satData['UrbanRemainRatio']=df_satData['Urban and built-up']/(df_satData['Area']-df_satData['Natural'])
#df_satData['UrbanRatio']=df_satData['UrbanRatio'].fillna(0)

In [13]:
modi_var = []
modi_var=['mean','sum','CropRatio', 'AvgUrbanNTL', 'UrbanRatio', 'CropRemainRatio', 'UrbanRemainRatio','Natural']
for i in ['Water',
 'Evergreen Needleleaf forest',
 'Evergreen Broadleaf forest',
 'Deciduous Needleleaf forest',
 'Deciduous Broadleaf forest',
 'Mixed forest',
 'Closed shrublands',
 'Open shrublands',
 'Woody savannas',
 'Savannas',
 'Grasslands',
 'Permanent wetlands',
 'Croplands',
 'Urban and built-up',
 'Cropland/Natural vegetation mosaic',
 'Snow and ice']:
    df_satData['mod_'+i]=df_satData[i]/df_satData['Area']
    modi_var.append('mod_'+i)
    modi_var.append(i)

## final dataframe from pca version of nl and modi 

In [14]:
df_modi_nlPca =  pd.concat([df_satData[modi_var],df_pcaNl],axis=1)

In [15]:
var = df_modi_nlPca.columns.tolist()

In [16]:
df_modi_nlPca[df_modi_nlPca<0] = 0

## finding feature ranking for a particular label

In [17]:
def featureRank(label):
    model = LogisticRegression(class_weight='balanced')
    selector = RFE(model,1, step=1)

    X = df_modi_nlPca.values
    X = preprocessing.scale(X)

    y = df_satData.loc[:,label].values

    selector.fit(X,y)

    selectedIdx = selector.get_support(indices=True)

    featureRanking = selector.ranking_

    ranked_var = pd.DataFrame(featureRanking,var)
    ranked_var = ranked_var.reset_index()
    ranked_var.columns = (['var','rank'])
    ranked_var = ranked_var.sort_values('rank')
    ranked_var.reset_index(inplace=True)
    del ranked_var['index']
    
    return ranked_var

## selecting features to maximize f1_score
#### Assumption: Set of first n features in feature ranking will produce maximum f1_score compared to any other set of n features.

In [18]:
def selectFeatures(ranked_var,label):
    metric_list = list()
    f1_list = list()
    for i in range(len(ranked_var)):
        var2 = ranked_var.loc[:i,'var'].values.tolist()

        X = df_modi_nlPca.loc[:,var2].values
        y = df_satData.loc[:,label].values

        model = LogisticRegression(class_weight='balanced')
        clf = make_pipeline(preprocessing.StandardScaler(),model)
        predicted = cross_validation.cross_val_predict(clf,X,y, cv=5)

        acc = metrics.accuracy_score(y,predicted)
        recall = metrics.recall_score(y,predicted)
        prec =  metrics.precision_score(y,predicted)

        cm = metrics.confusion_matrix(y, predicted)
        cm_score = float(cm[1][1])/(cm[1][1]+cm[1][0]+cm[0][1])

        f1 = metrics.f1_score(y,predicted)

        metric_list.append([f1,cm_score,acc,recall,prec])
        f1_list.append(f1)

    df_metric = pd.DataFrame(metric_list)
    df_metric.columns =  (['f1','cm','acc','recall','prec'])

    ## finding the global maximum index in f1_list
    global_maxima = f1_list.index(max(f1_list))

    ## plotting f1_score with features selected
    #figure()
    #plt.grid()
    #plt.plot(f1_list)
    #plt.title(label)
    #plt.xlabel('number of variables')
    #plt.ylabel('f1_score')
    
    ## finding indexes of all local maximums

    x = array(f1_list)
    # for local maxima
    local_maxima = argrelextrema(x, np.greater)[0].tolist()

    f1_acceptable = f1_list[global_maxima]-0.01

    i = 0
    while f1_list[i] < f1_acceptable:
        i += 1

    var_final = ranked_var.loc[:i,'var']

    X = df_modi_nlPca.loc[:,var_final].values
    y = df_satData.loc[:,label].values

    model = LogisticRegression()

    fit = model.fit(X,y)

    df_selecedVar = ranked_var.loc[:i].copy(deep=True)
    df_selecedVar.insert(2,'coeff',fit.coef_[0])

    return [df_selecedVar,f1_list[i],df_metric[i:i+1]]

In [20]:
select_feature = []
score_metric = []
for label in lis_labelsAll:
    ranked_var = featureRank(label)
    feature_and_score = selectFeatures(ranked_var,label)
    select_feature.append(feature_and_score)
    score_metric.append(feature_and_score[2].values[0])

In [21]:
df_scoreMetric = pd.DataFrame(score_metric,columns = ['f1','cm','acc','recall','prec'], index=lis_labelsAll)

In [22]:
df_scoreMetric.to_csv('scoreMetric_pcaModi.csv')

In [23]:
df_scoreMetric.sort_values(by='f1',ascending=False)

,f1,cm,acc,recall,prec
MSL_3,0.881159,0.787565,0.871875,0.856338,0.907463
BF_1,0.867153,0.765464,0.857812,0.886567,0.848571
FC_1,0.851371,0.741206,0.839063,0.862573,0.840456
MSW_3,0.826162,0.703812,0.842187,0.863309,0.792079
MSW_2,0.819320,0.693939,0.842187,0.874046,0.771044
FC_3,0.774026,0.631356,0.864062,0.837079,0.719807
MSL_1,0.772455,0.629268,0.881250,0.914894,0.668394
CHH_1,0.763393,0.617329,0.834375,0.822115,0.712500
EMP_3,0.760807,0.613953,0.870313,0.809816,0.717391
BF_3,0.738661,0.585616,0.810937,0.818182,0.673228


In [ ]:
X = df_modi_nlPca.copy(deep=True)
X.insert(0,'intercept',1.0)

cols = X.columns.tolist()
modi_cols = cols[:cols.index('pca_0')-2]
pca_cols = ['intercept'] + cols[cols.index('pca_0'):]

In [87]:
def find_summary(cols):
    X1 = X[cols].values
    y = df_satData.loc[:,label].values

    model = sm.Logit(y,X1)
    result = model.fit()
    summary = result.summary()

    df_summary = pd.DataFrame(result.params,result.pvalues)
    df_summary.reset_index(inplace=True)
    df_summary.index = cols
    df_summary.columns = ['param','pvals']
    
    return df_summary

In [88]:
df_summary = pd.concat([find_summary(modi_cols),find_summary(pca_cols)])

Optimization terminated successfully.
         Current function value: 0.248925
         Iterations 33
Optimization terminated successfully.
         Current function value: 0.375873
         Iterations 9


In [90]:
df_summary.sort_values(by='pvals',ascending=False)

,param,pvals
mod_Closed shrublands,2.049007e-01,1.133790e+02
mod_Savannas,1.590477e-01,5.413978e+01
mod_Urban and built-up,NaN,2.706123e+01
UrbanRatio,NaN,2.706120e+01
mod_Evergreen Needleleaf forest,3.181512e-02,2.535342e+01
mod_Permanent wetlands,3.626281e-02,1.931541e+01
mod_Open shrublands,1.345869e-01,9.466003e+00
UrbanRemainRatio,1.780559e-01,7.138844e+00
mod_Woody savannas,4.188503e-01,3.263419e+00
mod_Evergreen Broadleaf forest,4.681700e-01,2.558179e+00


In [100]:
example_label = select_feature[1][0].copy(deep=True)
example_label.index = example_label['var']
del example_label['var']

In [103]:
summary_example_label = pd.concat([example_label,df_summary],axis=1,join_axes=[example_label.index])

In [104]:
summary_example_label

,rank,coeff,param,pvals
var,,,,
UrbanRemainRatio,1,-4.549840e-22,0.178056,7.138844e+00
pca_2,2,-7.347586e-21,0.126680,-9.988316e-02
mod_Snow and ice,3,-6.111531e-23,NaN,NaN
CropRemainRatio,4,-3.654828e-21,0.092797,-2.745591e+00
pca_1,5,-1.171506e-20,0.000006,7.926679e-01
sum,6,-4.449677e-11,0.935736,-1.777307e-12
pca_5,7,-2.244026e-21,0.041683,7.638311e-01
